# COCO Image *Keypoints* Viewer
This notebook will allow you to view details about a COCO dataset and preview segmentations on annotated images.
Learn more about it at: http://cocodataset.org/

Note: Gist probably won't show the segmentations, but if you run this code in your own Jupyter Notebook, you'll see them.

The rest of the tutorial can be found at: http://www.immersivelimit.com/tutorials/create-coco-annotations-from-scratch

In [1]:
import IPython
import os
import json
import random
import numpy as np
import requests
from io import BytesIO
import base64
from math import trunc
from PIL import Image as PILImage
from PIL import ImageDraw as PILImageDraw

In [2]:
# Load the dataset json
class CocoDataset():
    def __init__(self, annotation_path, image_dir):
        self.annotation_path = annotation_path
        self.image_dir = image_dir
        self.colors = colors = ['blue', 'purple', 'red', 'green', 'orange', 'salmon', 'pink', 'gold',
                                'orchid', 'slateblue', 'limegreen', 'seagreen', 'darkgreen', 'olive',
                               'teal', 'aquamarine', 'steelblue', 'powderblue', 'dodgerblue', 'navy',
                               'magenta', 'sienna', 'maroon']
        
        json_file = open(self.annotation_path)
        self.coco = json.load(json_file)
        json_file.close()
        
        self.process_info()
        self.process_licenses()
        self.process_categories()
        self.process_images()
        self.process_segmentations()
        self.process_keypoints()
            
        
    def display_info(self):
        print('Dataset Info:')
        print('=============')
        for key, item in self.info.items():
            print('  {}: {}'.format(key, item))
        
        requirements = [['description', str],
                        ['url', str],
                        ['version', str],
                        ['year', int],
                        ['contributor', str],
                        ['date_created', str]]
        for req, req_type in requirements:
            if req not in self.info:
                print('ERROR: {} is missing'.format(req))
            elif type(self.info[req]) != req_type:
                print('ERROR: {} should be type {}'.format(req, str(req_type)))
        print('')

        
    def display_licenses(self):
        print('Licenses:')
        print('=========')
        
        requirements = [['id', int],
                        ['url', str],
                        ['name', str]]
        for license in self.licenses:
            for key, item in license.items():
                print('  {}: {}'.format(key, item))
            for req, req_type in requirements:
                if req not in license:
                    print('ERROR: {} is missing'.format(req))
                elif type(license[req]) != req_type:
                    print('ERROR: {} should be type {}'.format(req, str(req_type)))
            print('')
        print('')
        
    def display_categories(self):
        print('Categories:')
        print('=========')
        for sc_key, sc_val in self.super_categories.items():
            print('  super_category: {}'.format(sc_key))
            for cat_id in sc_val:
                print('    id {}: {}'.format(cat_id, self.categories[cat_id]['name']))
            print('')
            
    def display_keypoints(self, image_id, show_skelton=True, cmu_style=False, show_bbox=True, use_url=False):
        print('Image:')
        print('======')
        if image_id == 'random':
            image_id = random.choice(list(self.images.keys()))
        
        # Print the image info
        image = self.images[image_id]
        for key, val in image.items():
            print('  {}: {}'.format(key, val))
            
        # Open the image
        if use_url:
            image_path = image['coco_url']
            response = requests.get(image_path)
            image = PILImage.open(BytesIO(response.content))
            
        else:
            image_path = os.path.join(self.image_dir, image['file_name'])
            image = PILImage.open(image_path)
            
            buffer = BytesIO()
            image.save(buffer, format='PNG')
            buffer.seek(0)
            
            data_uri = base64.b64encode(buffer.read()).decode('ascii')
            image_path = "data:image/png;base64,{0}".format(data_uri)
            
        # Calculate the size and adjusted display size
        max_width = 600
        image_width, image_height = image.size
        adjusted_width = min(image_width, max_width)
        adjusted_ratio = adjusted_width / image_width
        adjusted_height = adjusted_ratio * image_height
        
        # Create skelton bones to be drawn
        skeltons = {}
        bbox_polygons = {}
        rle_regions = {}
        bone_colors = {}
        poly_colors = {}
        
        print('  keypoints ({}):'.format(len(self.keypoints[image_id])))
        for i, keyp in enumerate(self.keypoints[image_id]):
            skelton_keypoints = []
            
            for i, point_name in enumerate(self.categories[keyp['category_id']]['keypoints']):
                skelton_keypoints.append([keyp['keypoints'][i*3]*adjusted_ratio, keyp['keypoints'][i*3+1]*adjusted_ratio, keyp['keypoints'][i*3+2]])
                #print(' {} = [{}, {}], {} '.format(point_name, keyp['keypoints'][i*3], keyp['keypoints'][i*3+1], keyp['keypoints'][i*3+2]))  
            skeltons[keyp['id']] = skelton_keypoints
            
            if i < len(self.colors):
                poly_colors[keyp['id']] = self.colors[i]
            else:
                poly_colors[keyp['id']] = 'white'
                
            bbox = keyp['bbox']
            bbox_points = [bbox[0], bbox[1], bbox[0] + bbox[2], bbox[1],
                           bbox[0] + bbox[2], bbox[1] + bbox[3], bbox[0], bbox[1] + bbox[3],
                           bbox[0], bbox[1]]
            bbox_points = np.multiply(bbox_points, adjusted_ratio).astype(int)
            bbox_polygons[keyp['id']] = str(bbox_points).lstrip('[').rstrip(']')
            
            # Print details
            #print(' --> {}:{}:{}'.format(keyp['id'], poly_colors[keyp['id']], self.categories[keyp['category_id']]))    
            
        # Draw skeltons on image
        html  = '<div class="container" style="position:relative;">'
        html += '<img src="{}" style="position:relative;top:0px;left:0px;width:{}px;">'.format(image_path, adjusted_width)
        html += '<div class="svgclass"><svg width="{}" height="{}">'.format(adjusted_width, adjusted_height)
        
        if show_skelton:
            if cmu_style:
                for seg_id, skelton_keypoints in skeltons.items():
                    NUM_CMUPOINTS = 18
                    cmu_points = np.empty(NUM_CMUPOINTS, dtype=object)
                    cmu_points[0] = skelton_keypoints[0]   # nose
                    if(skelton_keypoints[5][2] != 0 and skelton_keypoints[6][2] != 0 ):
                        cmu_points[1] = [(skelton_keypoints[5][0] + skelton_keypoints[6][0])/2, (skelton_keypoints[5][1] + skelton_keypoints[6][1])/2, skelton_keypoints[5][2]] # center of shoulders
                    else:
                        cmu_points[1] = [0,0,0]
                    cmu_points[2] = skelton_keypoints[6]   # right_shoulder
                    cmu_points[3] = skelton_keypoints[8]   # right_elbow
                    cmu_points[4] = skelton_keypoints[10]  # right_wrist
                    cmu_points[5] = skelton_keypoints[5]   # left_shoulder
                    cmu_points[6] = skelton_keypoints[7]   # left_elbow
                    cmu_points[7] = skelton_keypoints[9]   # left_wrist
                    cmu_points[8] = skelton_keypoints[12]  # right_hip
                    cmu_points[9] = skelton_keypoints[14]  # right_knee
                    cmu_points[10] = skelton_keypoints[16] # right_ankle
                    cmu_points[11] = skelton_keypoints[11] # left_hip
                    cmu_points[12] = skelton_keypoints[13] # left_knee
                    cmu_points[13] = skelton_keypoints[15] # left_ankle
                    cmu_points[14] = skelton_keypoints[2]  # right_eye
                    cmu_points[15] = skelton_keypoints[1]  # right_ear
                    cmu_points[16] = skelton_keypoints[4]  # left_eye
                    cmu_points[17] = skelton_keypoints[3]  # left_ear
                    for i in range(len(cmu_points)):
                        if(cmu_points[i][2]==2):
                            html += '<circle cx="{}" cy="{}" r=4 fill="hsl({}, 100%, 50%)" fill-opacity="0.5" />'.format(cmu_points[i][0], cmu_points[i][1], 360/NUM_CMUPOINTS*i)          
                    NUM_CMUBONES = 17
                    cmu_bones = np.empty(NUM_CMUBONES, dtype=object)
                    cmu_bones[0] = [cmu_points[1], cmu_points[2]]
                    cmu_bones[1] = [cmu_points[1], cmu_points[5]]
                    cmu_bones[2] = [cmu_points[2], cmu_points[3]]
                    cmu_bones[3] = [cmu_points[3], cmu_points[4]]
                    cmu_bones[4] = [cmu_points[5], cmu_points[6]]
                    cmu_bones[5] = [cmu_points[6], cmu_points[7]]
                    cmu_bones[6] = [cmu_points[1], cmu_points[8]]
                    cmu_bones[7] = [cmu_points[8], cmu_points[9]]
                    cmu_bones[8] = [cmu_points[9], cmu_points[10]]
                    cmu_bones[9] = [cmu_points[1], cmu_points[11]]
                    cmu_bones[10] = [cmu_points[11], cmu_points[12]]
                    cmu_bones[11] = [cmu_points[12], cmu_points[13]]
                    cmu_bones[12] = [cmu_points[1], cmu_points[0]]
                    cmu_bones[13] = [cmu_points[0], cmu_points[14]]
                    cmu_bones[14] = [cmu_points[14], cmu_points[16]]
                    cmu_bones[15] = [cmu_points[0], cmu_points[15]]
                    cmu_bones[16] = [cmu_points[15], cmu_points[17]]             
                    for i, bone in enumerate(cmu_bones):
                        if(bone[0][2]!=0 and bone[1][2]!=0):
                            html += '<line x1="{}" y1="{}" x2="{}" y2="{}" style="stroke:hsl({}, 100%, 50%); stroke-width:4; stroke-opacity:0.5" />'.format(bone[0][0], bone[0][1], bone[1][0], bone[1][1], 360/NUM_CMUPOINTS*i)
                        
            else:
                for seg_id, skelton_keypoints in skeltons.items():
                    n = len(skelton_keypoints)
                    for i, point in enumerate(skelton_keypoints):
                        if(point[2]==2):
                            html += '<circle cx="{}" cy="{}" r=3 fill="hsl({}, 100%, 50%)" fill-opacity="0.5" />'.format(point[0], point[1], 360/n*i)               
                                
        if show_bbox:
            for seg_id, points in bbox_polygons.items():
                fill_color = poly_colors[seg_id]
                stroke_color = poly_colors[seg_id]
                html += '<polygon points="{}" style="fill:{}; stroke:{}; stroke-width:1; fill-opacity:0" />'.format(points, fill_color, stroke_color)
                
        html += '</svg></div>'
        html += '</div>'
        html += '<style>'
        html += '.svgclass { position:absolute; top:0px; left:0px;}'
        html += '</style>'
        return html
        
    def display_image(self, image_id, show_polys=True, show_bbox=True, show_crowds=True, use_url=False):
        print('Image:')
        print('======')
        if image_id == 'random':
            image_id = random.choice(list(self.images.keys()))
        
        # Print the image info
        image = self.images[image_id]
        for key, val in image.items():
            print('  {}: {}'.format(key, val))
            
        # Open the image
        if use_url:
            image_path = image['coco_url']
            response = requests.get(image_path)
            image = PILImage.open(BytesIO(response.content))
            
        else:
            image_path = os.path.join(self.image_dir, image['file_name'])
            image = PILImage.open(image_path)
            
            buffer = BytesIO()
            image.save(buffer, format='PNG')
            buffer.seek(0)
            
            data_uri = base64.b64encode(buffer.read()).decode('ascii')
            image_path = "data:image/png;base64,{0}".format(data_uri)
            
        # Calculate the size and adjusted display size
        max_width = 600
        image_width, image_height = image.size
        adjusted_width = min(image_width, max_width)
        adjusted_ratio = adjusted_width / image_width
        adjusted_height = adjusted_ratio * image_height
        
        # Create list of polygons to be drawn
        polygons = {}
        bbox_polygons = {}
        rle_regions = {}
        poly_colors = {}
        
        print('  segmentations ({}):'.format(len(self.segmentations[image_id])))
        for i, segm in enumerate(self.segmentations[image_id]):
            polygons_list = []
            if segm['iscrowd'] != 0:
                # Gotta decode the RLE
                px = 0
                x, y = 0, 0
                rle_list = []
                for j, counts in enumerate(segm['segmentation']['counts']):
                    if j % 2 == 0:
                        # Empty pixels
                        px += counts
                    else:
                        # Need to draw on these pixels, since we are drawing in vector form,
                        # we need to draw horizontal lines on the image
                        x_start = trunc(trunc(px / image_height) * adjusted_ratio)
                        y_start = trunc(px % image_height * adjusted_ratio)
                        px += counts
                        x_end = trunc(trunc(px / image_height) * adjusted_ratio)
                        y_end = trunc(px % image_height * adjusted_ratio)
                        if x_end == x_start:
                            # This is only on one line
                            rle_list.append({'x': x_start, 'y': y_start, 'width': 1 , 'height': (y_end - y_start)})
                        if x_end > x_start:
                            # This spans more than one line
                            # Insert top line first
                            rle_list.append({'x': x_start, 'y': y_start, 'width': 1, 'height': (image_height - y_start)})
                            
                            # Insert middle lines if needed
                            lines_spanned = x_end - x_start + 1 # total number of lines spanned
                            full_lines_to_insert = lines_spanned - 2
                            if full_lines_to_insert > 0:
                                full_lines_to_insert = trunc(full_lines_to_insert * adjusted_ratio)
                                rle_list.append({'x': (x_start + 1), 'y': 0, 'width': full_lines_to_insert, 'height': image_height})
                                
                            # Insert bottom line
                            rle_list.append({'x': x_end, 'y': 0, 'width': 1, 'height': y_end})
                if len(rle_list) > 0:
                    rle_regions[segm['id']] = rle_list  
            else:
                # Add the polygon segmentation
                for segmentation_points in segm['segmentation']:
                    segmentation_points = np.multiply(segmentation_points, adjusted_ratio).astype(int)
                    polygons_list.append(str(segmentation_points).lstrip('[').rstrip(']'))
            polygons[segm['id']] = polygons_list
            if i < len(self.colors):
                poly_colors[segm['id']] = self.colors[i]
            else:
                poly_colors[segm['id']] = 'white'
            
            bbox = segm['bbox']
            bbox_points = [bbox[0], bbox[1], bbox[0] + bbox[2], bbox[1],
                           bbox[0] + bbox[2], bbox[1] + bbox[3], bbox[0], bbox[1] + bbox[3],
                           bbox[0], bbox[1]]
            bbox_points = np.multiply(bbox_points, adjusted_ratio).astype(int)
            bbox_polygons[segm['id']] = str(bbox_points).lstrip('[').rstrip(']')
            
            # Print details
            #print('    {}:{}:{}'.format(segm['id'], poly_colors[segm['id']], self.categories[segm['category_id']]))
        
        
        
        # Draw segmentation polygons on image
        html  = '<div class="container" style="position:relative;">'
        html += '<img src="{}" style="position:relative;top:0px;left:0px;width:{}px;">'.format(image_path, adjusted_width)
        html += '<div class="svgclass"><svg width="{}" height="{}">'.format(adjusted_width, adjusted_height)
        
        if show_polys:
            for seg_id, points_list in polygons.items():
                fill_color = poly_colors[seg_id]
                stroke_color = poly_colors[seg_id]
                for points in points_list:
                    html += '<polygon points="{}" style="fill:{}; stroke:{}; stroke-width:1; fill-opacity:0.5" />'.format(points, fill_color, stroke_color)
        
        if show_crowds:
            for seg_id, rect_list in rle_regions.items():
                fill_color = poly_colors[seg_id]
                stroke_color = poly_colors[seg_id]
                for rect_def in rect_list:
                    x, y = rect_def['x'], rect_def['y']
                    w, h = rect_def['width'], rect_def['height']
                    html += '<rect x="{}" y="{}" width="{}" height="{}" style="fill:{}; stroke:{}; stroke-width:1; fill-opacity:0.5; stroke-opacity:0.5" />'.format(x, y, w, h, fill_color, stroke_color)
            
        if show_bbox:
            for seg_id, points in bbox_polygons.items():
                fill_color = poly_colors[seg_id]
                stroke_color = poly_colors[seg_id]
                html += '<polygon points="{}" style="fill:{}; stroke:{}; stroke-width:1; fill-opacity:0" />'.format(points, fill_color, stroke_color)
                
        html += '</svg></div>'
        html += '</div>'
        html += '<style>'
        html += '.svgclass { position:absolute; top:0px; left:0px;}'
        html += '</style>'
        return html
       
    def process_info(self):
        self.info = self.coco['info']
    
    def process_licenses(self):
        self.licenses = self.coco['licenses']
    
    def process_categories(self):
        self.categories = {}
        self.super_categories = {}
        for category in self.coco['categories']:
            cat_id = category['id']
            super_category = category['supercategory']
            
            # Add category to the categories dict
            if cat_id not in self.categories:
                self.categories[cat_id] = category
            else:
                print("ERROR: Skipping duplicate category id: {}".format(category))

            # Add category to super_categories dict
            if super_category not in self.super_categories:
                self.super_categories[super_category] = {cat_id} # Create a new set with the category id
            else:
                self.super_categories[super_category] |= {cat_id} # Add category id to the set
                
    def process_images(self):
        self.images = {}
        for image in self.coco['images']:
            image_id = image['id']
            if image_id in self.images:
                print("ERROR: Skipping duplicate image id: {}".format(image))
            else:
                self.images[image_id] = image
                
    def process_segmentations(self):
        self.segmentations = {}
        for segmentation in self.coco['annotations']:
            image_id = segmentation['image_id']
            if image_id not in self.segmentations:
                self.segmentations[image_id] = []
            self.segmentations[image_id].append(segmentation)
                
    def process_keypoints(self):
        self.keypoints = {}
        for keypoint in self.coco['annotations']:
            image_id = keypoint['image_id']
            if image_id not in self.keypoints:
                self.keypoints[image_id] = []
            self.keypoints[image_id].append(keypoint)

In [3]:
#annotation_path = '/home/coco-keypoints/akTwelve/sample_keypoints.json'
annotation_path = '/home/coco-keypoints/annotations/person_keypoints_val2017.json'
image_dir = '/home/coco-keypoints/akTwelve/images'

coco_dataset = CocoDataset(annotation_path, image_dir)
coco_dataset.display_info()
coco_dataset.display_licenses()
coco_dataset.display_categories()

Dataset Info:
  description: COCO 2017 Dataset
  url: http://cocodataset.org
  version: 1.0
  year: 2017
  contributor: COCO Consortium
  date_created: 2017/09/01

Licenses:
  url: http://creativecommons.org/licenses/by-nc-sa/2.0/
  id: 1
  name: Attribution-NonCommercial-ShareAlike License

  url: http://creativecommons.org/licenses/by-nc/2.0/
  id: 2
  name: Attribution-NonCommercial License

  url: http://creativecommons.org/licenses/by-nc-nd/2.0/
  id: 3
  name: Attribution-NonCommercial-NoDerivs License

  url: http://creativecommons.org/licenses/by/2.0/
  id: 4
  name: Attribution License

  url: http://creativecommons.org/licenses/by-sa/2.0/
  id: 5
  name: Attribution-ShareAlike License

  url: http://creativecommons.org/licenses/by-nd/2.0/
  id: 6
  name: Attribution-NoDerivs License

  url: http://flickr.com/commons/usage/
  id: 7
  name: No known copyright restrictions

  url: http://www.usa.gov/copyright.shtml
  id: 8
  name: United States Government Work


Categories:
  su

In [10]:
html = coco_dataset.display_image(409475, use_url=True)
IPython.display.HTML(html)

Image:
  license: 4
  file_name: 000000409475.jpg
  coco_url: http://images.cocodataset.org/val2017/000000409475.jpg
  height: 480
  width: 640
  date_captured: 2013-11-20 07:11:56
  flickr_url: http://farm4.staticflickr.com/3541/3404640306_5487b904da_z.jpg
  id: 409475
  segmentations (2):


In [9]:
html = coco_dataset.display_keypoints(409475, cmu_style=True, use_url=True)
IPython.display.HTML(html)

Image:
  license: 4
  file_name: 000000409475.jpg
  coco_url: http://images.cocodataset.org/val2017/000000409475.jpg
  height: 480
  width: 640
  date_captured: 2013-11-20 07:11:56
  flickr_url: http://farm4.staticflickr.com/3541/3404640306_5487b904da_z.jpg
  id: 409475
  keypoints (2):


# Sample Annotations JSON
Below are the contents of 'sample_annotations.json'.

In [6]:
with open(annotation_path) as json_file:
    lines = json_file.readlines()
for line in lines:
    print(line)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)

